In [60]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [61]:
# Step 1: Load the master (already processed) file and new weekly file
df_old_processed = pd.read_excel('/content/gdrive/MyDrive/FirstTask/visits_with_Location5.xlsx')  # already labeled
df_new_all = pd.read_excel('/content/gdrive/MyDrive/FirstTask/visits.xlsx')  # includes old + new data

# Step 2: District
df_coordinations = pd.read_excel('/content/gdrive/MyDrive/FirstTask/coordinates.xlsx',sheet_name="2")
df_coordinations = df_coordinations.loc[:, ["xx", "yy", "اسم الحي باللغة العربية", "اسم المحافظة باللغة العربية", "اسم المنطقة باللغة العربية"]]
districts_data = df_coordinations.rename({
    "xx": "longitude",
    "yy": "latitude",
    "اسم الحي باللغة العربية": "name"
}, axis="columns")

# Step 3: Extract latitude and longitude from the coordinates column
df_new_all[['latitude', 'longitude']] = df_new_all['الاحداثيات'].str.split(',', expand=True).astype(float)

# Step 4: Detect new (unprocessed) rows using رقم الزيارة
df_new = df_new_all[~df_new_all['رقم الزيارة'].isin(df_old_processed['رقم الزيارة'])].copy()


In [62]:
# Step 5: Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Step 6: Labeling function
def add_closest_district_info_to_target_df(districts, target_df):
    def get_closest_district_info(target_lon, target_lat):
        distances = districts.apply(
            lambda row: haversine(target_lat, target_lon, row['latitude'], row['longitude']),
            axis=1
        )
        closest_idx = distances.idxmin()
        closest = districts.loc[closest_idx]
        return closest[['name', 'اسم المحافظة باللغة العربية', 'اسم المنطقة باللغة العربية']]

    def apply_closest(row):
        if pd.isna(row['longitude']) or pd.isna(row['latitude']):
            return pd.Series([None, None, None], index=['name', 'اسم المحافظة باللغة العربية', 'اسم المنطقة باللغة العربية'])
        else:
            return get_closest_district_info(row['longitude'], row['latitude'])

    results = target_df.apply(apply_closest, axis=1)
    return pd.concat([target_df, results], axis=1)

In [66]:
# Step 7: Apply the function to label only the new rows
df_new_labeled = add_closest_district_info_to_target_df(districts_data, df_new)

# Step 8: Rename columns
df_new_labeled = df_new_labeled.rename({
    "name": "الحي الأقرب لزيارة",
    "اسم المحافظة باللغة العربية": "محافظة الحي الأقرب",
    "اسم المنطقة باللغة العربية": "منطقة الحي الأقرب"
}, axis="columns")



In [80]:
df_new_labeled.head()

,رقم الزيارة,معرف المخالفة,نوع الزيارة,حالة الزيارة,تاريخ انشاء الزيارة,تاريخ انهاء الزيارة,الاحداثيات,المنطقة,المحافظة,القطاع,...,سبب القرار,تاريخ اعتماد المخالفة,رقم بند اللائحة,وصف بند اللائحة,هل المخالفة انذار,latitude,longitude,الحي الأقرب لزيارة,محافظة الحي الأقرب,منطقة الحي الأقرب
14,NaN,V-3439322,زيارة غير مجدولة,تم ارسالها للنظر والفصل,3/10/25 2:43:08 PM,3/10/25 2:49:56 PM,"26.4873436,50.0376434",الشرقية,مدينة الدمام,شركة المياه الوطنية,...,NaN,2025-03-18 20:21,57/1,تدفق المياه خارج المباني العامة أو الخاصة,لا,26.487344,50.037643,الديرة2,القطيف,المنطقة الشرقية


In [81]:
# Step 9: Combine new + old
df_combined = pd.concat([df_old_processed, df_new_labeled], ignore_index=True)
df_combined

,Unnamed: 0,رقم الزيارة,معرف المخالفة,نوع الزيارة,حالة الزيارة,تاريخ انشاء الزيارة,تاريخ انهاء الزيارة,الاحداثيات,المنطقة,المحافظة,...,latitude,longitude,الحي الأقرب لزيارة,محافظة الحي الأقرب,منطقة الحي الأقرب,Visit_Start_Date,Visit_Start_Time,Visit_End_Date,Visit_End_Time,visit_from_baladi
0,0.0,SWA-10001,V-3159144,زيارة غير مجدولة,لا يوجد بيانات مخالف,2025-01-19 10:38:57,2025-01-19 11:03:21,"21.662353897513967,39.20903986489302",منطقة مكة المكرمة,محافظة جدة,...,21.662354,39.209040,بريمان,جدة,منطقة مكة المكرمة,2025-01-19,10:38:57,2025-01-19,11:03:21,0.0
1,1.0,SWA-10004,V-3160208,زيارة غير مجدولة,تم ارسالها للنظر والفصل,2025-01-19 12:45:34,2025-01-19 13:01:59,"25.535039574167218,49.60909801462924",المنطقة الشرقية,محافظة الإحساء,...,25.535040,49.609098,الإسكان2,الاحساء,المنطقة الشرقية,2025-01-19,12:45:34,2025-01-19,13:01:59,0.0
2,2.0,SWA-10004,V-3160208,زيارة غير مجدولة,تم ارسالها للنظر والفصل,2025-01-19 12:45:34,2025-01-19 13:01:59,"25.535039574167218,49.60909801462924",المنطقة الشرقية,محافظة الإحساء,...,25.535040,49.609098,الإسكان2,الاحساء,المنطقة الشرقية,2025-01-19,12:45:34,2025-01-19,13:01:59,0.0
3,3.0,SWA-10006,V-3160696,زيارة غير مجدولة,تم ارسالها للنظر والفصل,2025-01-19 14:23:51,2025-01-19 14:49:48,"25.545890862332147,49.60578741955767",المنطقة الشرقية,محافظة الإحساء,...,25.545891,49.605787,الإسكان2,الاحساء,المنطقة الشرقية,2025-01-19,14:23:51,2025-01-19,14:49:48,0.0
4,4.0,SWA-10006,V-3160696,زيارة غير مجدولة,تم ارسالها للنظر والفصل,2025-01-19 14:23:51,2025-01-19 14:49:48,"25.545890862332147,49.60578741955767",المنطقة الشرقية,محافظة الإحساء,...,25.545891,49.605787,الإسكان2,الاحساء,المنطقة الشرقية,2025-01-19,14:23:51,2025-01-19,14:49:48,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21813,21813.0,SWA-94204,NaN,زيارة غير مجدولة,بانتظار التفتيش,2025-07-07 18:40:33,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2025-07-07,18:40:33,NaT,NaN,0.0
21814,21814.0,SWA-94205,NaN,زيارة غير مجدولة,بانتظار المراجعة والاعتماد,2025-07-07 20:25:38,2025-07-07 20:34:59,"26.29920581806543,50.20552534658603",المنطقة الشرقية,محافظة الخبر,...,26.299206,50.205525,مدينة العمال,الخبر,المنطقة الشرقية,2025-07-07,20:25:38,2025-07-07,20:34:59,0.0
21815,21815.0,SWA-94206,NaN,زيارة غير مجدولة,بانتظار المراجعة والاعتماد,2025-07-07 21:09:03,2025-07-07 21:28:44,"26.30173991437687,50.20118972350996",المنطقة الشرقية,محافظة الخبر,...,26.301740,50.201190,العقربية,الخبر,المنطقة الشرقية,2025-07-07,21:09:03,2025-07-07,21:28:44,0.0
21816,21816.0,SWA-95001,NaN,بلاغ مواطن 940,مكتملة - ممتثل,2025-07-06 11:22:37,2025-07-07 12:46:45,"25.367171854077586,49.572906189252386",المنطقة الشرقية,محافظة الإحساء,...,25.367172,49.572906,المزروعية,الاحساء,المنطقة الشرقية,2025-07-06,11:22:37,2025-07-07,12:46:45,1.0


In [99]:
# check
visit_id = df_old_processed['رقم الزيارة'].sample(1).values[0]

display(df_old_processed[df_old_processed['رقم الزيارة'] == visit_id][['رقم الزيارة', 'الحي الأقرب لزيارة']])

display(df_combined[df_combined['رقم الزيارة'] == visit_id][['رقم الزيارة', 'الحي الأقرب لزيارة']])


,رقم الزيارة,الحي الأقرب لزيارة
16450,SWA-63090,المروج


,رقم الزيارة,الحي الأقرب لزيارة
16450,SWA-63090,المروج


In [100]:
matching_rows = df_new_labeled[df_new_labeled['رقم الزيارة'].isin(df_combined['رقم الزيارة'])]
display(matching_rows.head())

,رقم الزيارة,معرف المخالفة,نوع الزيارة,حالة الزيارة,تاريخ انشاء الزيارة,تاريخ انهاء الزيارة,الاحداثيات,المنطقة,المحافظة,القطاع,...,سبب القرار,تاريخ اعتماد المخالفة,رقم بند اللائحة,وصف بند اللائحة,هل المخالفة انذار,latitude,longitude,الحي الأقرب لزيارة,محافظة الحي الأقرب,منطقة الحي الأقرب
14,NaN,V-3439322,زيارة غير مجدولة,تم ارسالها للنظر والفصل,3/10/25 2:43:08 PM,3/10/25 2:49:56 PM,"26.4873436,50.0376434",الشرقية,مدينة الدمام,شركة المياه الوطنية,...,NaN,2025-03-18 20:21,57/1,تدفق المياه خارج المباني العامة أو الخاصة,لا,26.487344,50.037643,الديرة2,القطيف,المنطقة الشرقية
